In [403]:
import sqlite3
from sqlalchemy import create_engine 
import re
import requests
import pandas as pd # dataframes
import numpy as np
from math import pi
import datetime
from itertools import chain
from collections import namedtuple
from bokeh.io import show, output_notebook
from bokeh.models import GuideRenderer
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_file, show
from bokeh.models import HoverTool, FuncTickFormatter, FixedTicker, ColumnDataSource, SingleIntervalTicker, LinearAxis, Legend


In [404]:
#reading csv into pandas dataframe for InjectionWells
injection_wells = pd.read_csv("data/InjectionWells.csv")

In [405]:
#reading csv into pandas dataframe for OkQuakes
ok_quakes = pd.read_csv("data/okQuakes.csv")

In [406]:
#code to look at datatypes of a dataframe
injection_wells.dtypes

API#             float64
Operator          object
Operator ID      float64
WellType          object
WellName          object
WellNumber        object
OrderNumbers     float64
Approval Date     object
County            object
Sec               object
Twp               object
Rng               object
QQQQ              object
LAT              float64
LONG             float64
PSI               object
BBLS              object
ZONE              object
Unnamed: 18      float64
Unnamed: 19      float64
Unnamed: 20      float64
dtype: object

In [407]:
#code to look at datatypes of a dataframe
ok_quakes.dtypes

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

In [408]:
#code to look at the set up of the dataframe
injection_wells.head()

,API#,Operator,Operator ID,WellType,WellName,WellNumber,OrderNumbers,Approval Date,County,Sec,...,Rng,QQQQ,LAT,LONG,PSI,BBLS,ZONE,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,3.500300e+09,PHOENIX PETROCORP INC,19499.0,2R,SE EUREKA UNIT-TUCKER #1,21,133856.0,9/6/1977,ALFALFA,13,...,10W,C-SE SE,36.900324,-98.21826,"2,500",300,CHEROKEE,NaN,NaN,NaN
1,3.500300e+09,CHAMPLIN EXPLORATION INC,4030.0,2R,CHRISTENSEN,1,470258.0,11/27/2002,ALFALFA,21,...,09W,C-NW NW,36.896636,-98.17772,"2,400","1,000",RED FORK,NaN,NaN,NaN
2,3.500321e+09,LINN OPERATING INC,22182.0,2R,NE CHEROKEE UNIT,85,329426.0,8/19/1988,ALFALFA,24,...,11W,SE NE,36.806113,-98.32584,"1,050","1,000",RED FORK,NaN,NaN,NaN
3,3.500321e+09,SANDRIDGE EXPLORATION & PRODUCTION LLC,22281.0,2R,VELMA,2-19,281652.0,7/11/1985,ALFALFA,19,...,10W,SW NE NE SW,36.888589,-98.31853,"3,152","1,000",RED FORK,NaN,NaN,NaN
4,3.500321e+09,CHAMPLIN EXPLORATION INC,4030.0,2R,GRAY,1A,470257.0,11/27/2002,ALFALFA,20,...,09W,SE SW NW,36.892128,-98.19462,"1,000","2,400",RED FORK,NaN,NaN,NaN


In [409]:
#code to look at the set up of the dataframe
ok_quakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1973-03-17T07:43:05.500Z,36.087,-106.168,6.0,4.5,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:10.078Z,New Mexico,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
1,1973-05-25T14:40:13.900Z,33.917,-90.775,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:12.859Z,Mississippi,earthquake,NaN,NaN,NaN,NaN,reviewed,s,us
2,1973-09-19T13:28:20.500Z,37.160,-104.594,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:20.295Z,Colorado,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
3,1973-09-23T03:58:54.900Z,37.148,-104.571,5.0,4.2,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:20.346Z,Colorado,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
4,1974-02-15T13:33:49.200Z,36.500,-100.693,24.0,4.5,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:22.859Z,Oklahoma,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us


In [410]:
#changing time column to a datetime datatype in ok_quakes dataset
ok_quakes['time'] = pd.to_datetime(ok_quakes['time'], format='%Y-%m-%dT%H:%M:%S.%fZ')
ok_quakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1973-03-17 07:43:05.500,36.087,-106.168,6.0,4.5,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:10.078Z,New Mexico,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
1,1973-05-25 14:40:13.900,33.917,-90.775,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:12.859Z,Mississippi,earthquake,NaN,NaN,NaN,NaN,reviewed,s,us
2,1973-09-19 13:28:20.500,37.160,-104.594,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:20.295Z,Colorado,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
3,1973-09-23 03:58:54.900,37.148,-104.571,5.0,4.2,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:20.346Z,Colorado,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
4,1974-02-15 13:33:49.200,36.500,-100.693,24.0,4.5,mb,NaN,NaN,NaN,NaN,...,2014-11-06T23:21:22.859Z,Oklahoma,earthquake,NaN,NaN,NaN,NaN,reviewed,us,us


In [411]:
#changing Approval Date column to a datetime datatype in Injection wells dataset
injection_wells['Approval Date'] = pd.to_datetime(injection_wells['Approval Date'], format='%m/%d/%Y')

In [412]:
#Creating the cursor connections
conn = sqlite3.connect('injection_wells.db')
cur = conn.cursor() 
cursor = conn.cursor()

In [413]:
#Creating the database Injection_Wells
injection_wells.to_sql("Injection_Wells", conn, if_exists='replace', index=True)

C:\Users\HP\Anaconda3\lib\site-packages\pandas\core\generic.py:1362: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


In [414]:
#Creating the database OK_Quakes
ok_quakes.to_sql("OK_Quakes", conn, if_exists='replace', index=True)

In [415]:
#tesing new table from query

cur.execute("pragma table_info(OK_Quakes);")
tablecolumnnames=cur.fetchall()
print(tablecolumnnames)

[(0, 'index', 'INTEGER', 0, None, 0), (1, 'time', 'TIMESTAMP', 0, None, 0), (2, 'latitude', 'REAL', 0, None, 0), (3, 'longitude', 'REAL', 0, None, 0), (4, 'depth', 'REAL', 0, None, 0), (5, 'mag', 'REAL', 0, None, 0), (6, 'magType', 'TEXT', 0, None, 0), (7, 'nst', 'REAL', 0, None, 0), (8, 'gap', 'REAL', 0, None, 0), (9, 'dmin', 'REAL', 0, None, 0), (10, 'rms', 'REAL', 0, None, 0), (11, 'net', 'TEXT', 0, None, 0), (12, 'id', 'TEXT', 0, None, 0), (13, 'updated', 'TEXT', 0, None, 0), (14, 'place', 'TEXT', 0, None, 0), (15, 'type', 'TEXT', 0, None, 0), (16, 'horizontalError', 'REAL', 0, None, 0), (17, 'depthError', 'REAL', 0, None, 0), (18, 'magError', 'REAL', 0, None, 0), (19, 'magNst', 'REAL', 0, None, 0), (20, 'status', 'TEXT', 0, None, 0), (21, 'locationSource', 'TEXT', 0, None, 0), (22, 'magSource', 'TEXT', 0, None, 0)]


In [416]:
#Drop table command if it already exists
cur.execute("DROP TABLE Quake_Year;")

In [417]:
#sql query to create new Quake_Year table from OK_Quakes table with query 
cur.execute("create table Quake_Year as select time, strftime('%Y', time) as quake_year from OK_Quakes where type='earthquake' and place like '%Ok%';")


In [418]:
cur.execute("select * from Quake_Year ")
results=cur.fetchall()
print(results)

[('1974-02-15 13:33:49.200000', '1974'), ('1974-12-16 02:30:21.400000', '1974'), ('1975-09-13 01:25:02.800000', '1975'), ('1975-10-12 02:58:11.200000', '1975'), ('1975-11-29 14:29:40.900000', '1975'), ('1976-03-30 09:27:01', '1976'), ('1976-04-16 18:59:44.200000', '1976'), ('1976-04-19 04:42:42.200000', '1976'), ('1979-06-07 07:39:35.600000', '1979'), ('1979-09-13 00:49:19.800000', '1979'), ('1980-11-02 10:00:49.300000', '1980'), ('1981-07-11 21:09:22.540000', '1981'), ('1982-05-03 07:54:50', '1982'), ('1982-12-10 09:23:48.980000', '1982'), ('1982-12-31 08:17:37.350000', '1982'), ('1983-03-11 16:50:45.310000', '1983'), ('1984-01-06 17:14:49.800000', '1984'), ('1984-02-03 04:38:27.110000', '1984'), ('1984-03-03 11:42:02.400000', '1984'), ('1984-11-20 10:57:32.020000', '1984'), ('1985-09-23 01:03:43.060000', '1985'), ('1985-11-07 08:35:33.320000', '1985'), ('1985-12-31 18:27:25.820000', '1985'), ('1986-02-14 06:09:04.720000', '1986'), ('1986-02-24 23:52:22.030000', '1986'), ('1986-04-05 

In [419]:
#tesing new Quake_Year table from query

cur.execute("pragma table_info(Quake_Year);")
tablecolumnnames=cur.fetchall()
print(tablecolumnnames)

[(0, 'time', 'NUM', 0, None, 0), (1, 'quake_year', '', 0, None, 0)]


In [420]:
#sql query of Quake_Year table for Quake_year_count list
cur.execute("select quake_year, count(quake_year) as quake_year_count from Quake_Year group by quake_year;")
quake_year_count=cur.fetchall()
print(quake_year_count)

[('1974', 2), ('1975', 3), ('1976', 3), ('1979', 2), ('1980', 1), ('1981', 1), ('1982', 3), ('1983', 1), ('1984', 4), ('1985', 3), ('1986', 23), ('1987', 22), ('1988', 17), ('1989', 5), ('1990', 3), ('1991', 1), ('1992', 14), ('1993', 3), ('1994', 2), ('1995', 6), ('1996', 1), ('1997', 2), ('1998', 3), ('1999', 1), ('2000', 1), ('2002', 5), ('2003', 2), ('2004', 5), ('2005', 3), ('2006', 9), ('2007', 6), ('2008', 11), ('2009', 50), ('2010', 184), ('2011', 166), ('2012', 92), ('2013', 324), ('2014', 2041), ('2015', 2863), ('2016', 1969)]


In [421]:
#Drop table command if it already exists
cur.execute("DROP TABLE Year;")

In [422]:
#sql query to create new Year table from from Injection Well table with query
cur.execute("create table Year as select  WellName,strftime('%Y', `Approval Date`) as approval_year from Injection_Wells;")


    
  

In [423]:
#tesing new table from query

cur.execute("pragma table_info(Year);")
tablecolumnnames=cur.fetchall()
print(tablecolumnnames)

[(0, 'WellName', 'TEXT', 0, None, 0), (1, 'approval_year', '', 0, None, 0)]


In [424]:
#sql query of year table for year count list
cur.execute("select approval_year, count(approval_year) as year_count from Year group by approval_year;")
year_count=cur.fetchall()
print(year_count)

[(None, 0), ('1936', 1), ('1945', 1), ('1946', 1), ('1947', 4), ('1948', 1), ('1949', 8), ('1950', 13), ('1951', 2), ('1952', 8), ('1953', 34), ('1954', 34), ('1955', 45), ('1956', 64), ('1957', 71), ('1958', 65), ('1959', 179), ('1960', 126), ('1961', 157), ('1962', 134), ('1963', 95), ('1964', 64), ('1965', 96), ('1966', 80), ('1967', 92), ('1968', 276), ('1969', 95), ('1970', 99), ('1971', 145), ('1972', 192), ('1973', 79), ('1974', 77), ('1975', 76), ('1976', 77), ('1977', 79), ('1978', 76), ('1979', 123), ('1980', 142), ('1981', 132), ('1982', 173), ('1983', 242), ('1984', 243), ('1985', 141), ('1986', 182), ('1987', 199), ('1988', 175), ('1989', 159), ('1990', 268), ('1991', 195), ('1992', 254), ('1993', 208), ('1994', 143), ('1995', 190), ('1996', 212), ('1997', 175), ('1998', 177), ('1999', 134), ('2000', 175), ('2001', 153), ('2002', 167), ('2003', 194), ('2004', 170), ('2005', 165), ('2006', 200), ('2007', 275), ('2008', 234), ('2009', 246), ('2010', 293), ('2011', 438), ('20

In [425]:
#turning quake year count list into dataframe and changing the column names and testing it worked correctly

df2 = pd.DataFrame(quake_year_count)
df2.columns = ['QuakeYear', 'QuakeCount']

print(df2)

   QuakeYear  QuakeCount
0       1974           2
1       1975           3
2       1976           3
3       1979           2
4       1980           1
5       1981           1
6       1982           3
7       1983           1
8       1984           4
9       1985           3
10      1986          23
11      1987          22
12      1988          17
13      1989           5
14      1990           3
15      1991           1
16      1992          14
17      1993           3
18      1994           2
19      1995           6
20      1996           1
21      1997           2
22      1998           3
23      1999           1
24      2000           1
25      2002           5
26      2003           2
27      2004           5
28      2005           3
29      2006           9
30      2007           6
31      2008          11
32      2009          50
33      2010         184
34      2011         166
35      2012          92
36      2013         324
37      2014        2041
38      2015        2863


In [426]:
#turning year count list into dataframe and changing the column names and testing it worked correctly

df = pd.DataFrame(year_count)
df.columns = ['Year', 'Count']

df.head()

,Year,Count
0,None,0
1,1936,1
2,1945,1
3,1946,1
4,1947,4


In [428]:
#Bokeh visualization
#Pull the year and count of year into their own list to pass to bokeh for the different axis and plot values.
#Injection wells
count = [row[1] for row in year_count]
year = [row[0] for row in year_count]

#OK Quakes
qcount = [row[1] for row in quake_year_count]
qyear = [row[0] for row in quake_year_count]

#Set to open charts in the notebook
output_notebook()





p = figure(plot_width=1200, plot_height=600, x_axis_type=None, y_axis_type=None)

p.line(x=year, y=count, color="#f9a602", legend="Injection Wells", line_width=2)
p.line(x=qyear, y=qcount, color="navy", legend="EarthQuakes", line_width=2)
#p.multi_line(xs=[year, qyear], ys= [count, qcount], color=["firebrick", "navy"], alpha=[1.8, 1.3], line_width=2)

#formatting the y and x axis start ticks
p.y_range.start = 0
p.x_range.start = 1936
p.x_range.end = 2016



p.legend.location= 'top_left'

#formatting the x axis ticks and tick labels
xticker = SingleIntervalTicker(interval=2, num_minor_ticks=2)
xaxis = LinearAxis(ticker=xticker)
p.add_layout(xaxis, 'below')
p.xaxis.major_label_orientation = "vertical"


#formatting the y axis left ticks and tick labels
yticker = SingleIntervalTicker(interval=200, num_minor_ticks=2)
yaxis = LinearAxis(ticker=yticker)
p.add_layout(yaxis, 'left')

show(p)


Loading BokehJS ...